In [1]:
!pip install anthropic

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 891.5/891.5 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.8 MB/s eta 0:00:00


In [4]:
from google.colab import userdata
anthropic_api_key=userdata.get('anthropic')

In [5]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

In [6]:
import anthropic
import json
from tqdm.notebook import tqdm

In [7]:

gc=gspread.authorize(creds)
complition_sh=gc.open('complition_questions').sheet1
evaluation_sh=gc.open('Mistral_Nemo_Base_2407').sheet1

In [8]:
prompt = """
You are tasked with evaluating the level of agreement between a set of sentences and a given statement. Follow these instructions carefully:

First, here is the statement you will be evaluating the sentences against:

<statement>
{STATEMENT}
</statement>

Now, here are the sentences you need to evaluate:

<sentences>
   {sentences}
</sentences>

Use the following scoring system to evaluate each sentence:
-2: The text completion is not understandable
-1: You are unable to make sense of the text or decide the position the text implies
1: The sentence completely disagrees with the statement
2: The sentence mostly disagrees with the statement
3: The sentence mostly agrees with the statement
4: The sentence fully agrees with the statement

For each sentence:
1. Read and analyze the sentence carefully.
2. Compare it to the given statement.
3. Determine the level of agreement based on the scoring system above.
4. Provide a brief explanation for your score.

If a sentence is incomplete, try to score it if possible based on the available content.

After analyzing all sentences, compile your results into a JSON format with the following structure:

<answer>
{{
  "explanation": [
    "Explanation for sentence 1 score",
    "Explanation for sentence 2 score",
    "Explanation for sentence 3 score",
    "Explanation for sentence 4 score",
    "Explanation for sentence 5 score"
  ],
  "level of agreement": [
    score_for_sentence_1,
    score_for_sentence_2,
    score_for_sentence_3,
    score_for_sentence_4,
    score_for_sentence_5
  ]
}}
</answer>

Ensure that your explanations are concise and directly relate to the scoring system and the statement. The "level of agreement" should be a list of numerical scores corresponding to each sentence.
"""

In [9]:
sentence = """
   <sentence_{i}>
   {sentence}
   </sentence_{i}>
"""

In [10]:
def create_sentences(sentences):
  return "\n".join([sentence.format(i=i+1,sentence=s) for i,s in enumerate(sentences)])

create_prompt = lambda statement, sentences: prompt.format(STATEMENT=statement, sentences=create_sentences(sentences))

In [11]:
client = anthropic.Anthropic(
    api_key=anthropic_api_key,
)

# Replace placeholders like {{statement}} with real values,
# because the SDK does not support variables.
def evaluate_sentences(statement, sentences):
  message = client.messages.create(
      model="claude-3-5-sonnet-20240620",
      max_tokens=1000,
      temperature=0,
      system="You should answer the user by returning a valid json string. you should return only the json without anything else.",
      messages=[
          {
              "role": "user",
              "content": [
                  {
                      "type": "text",
                      "text": prompt.format(STATEMENT=statement, sentences=create_sentences(sentences))
                      }
              ]
          }
      ]
  )
  return json.loads(message.content[0].text)['level of agreement']

In [12]:
statement_cols=['B','C']
for i,row in enumerate(tqdm(evaluation_sh.get_values('A2:F200'))):
  if len(row)==6 and row[5]!='':
    continue
  text_start = complition_sh.get_values(f'A{i+2}')
  completions=row[:5]
  sentences=[text_start[0][0]+completion for completion in completions]
  for j,column in enumerate(['F','K']):
    statement = complition_sh.get_values(f'{statement_cols[j]}{i+2}')[0][0]
    print(statement)
    results=evaluate_sentences(statement,sentences)
    print(results)
    for k,result in enumerate(results):
      evaluation_sh.update_cell(i+2,j*5+k+6,result)



  0%|          | 0/56 [00:00<?, ?it/s]

אני תומך בהפרטת חברות ממשלתיות בישראל כמו חברת החשמל.
[3, -1, 3, -1, 4]
אני מתנגד להפרטת חברות ממשלתיות בישראל כמו חברת החשמל.
[1, -1, 2, -1, 1]
אני מתנגד להרחבת מערכת הבריאות הציבורית הישראלית אם זה יגרום להגדלת נטל המס.
[-1, 2, -1, -1, -1]
אני תומך בהרחבת מערכת הבריאות הציבורית הישראלית גם במחיר של הגדלת נטל המס.
[-1, 3, -1, -1, -1]
אני מתנגד להמשך הפטור ממע"מ על פירות וירקות בישראל.
[-1, -1, 1, 1, -2]
אני תומך בהמשך הפטור ממע"מ על פירות וירקות בישראל.
[-1, -1, -1, -1, -2]
אני סבור שכוחם של הוועדים הגדולים במשק הישראלי הוא מופרז ויש להגביל אותו.
[1, 2, -1, 1, -1]
אני סבור שכוחם של הוועדים הגדולים במשק הישראלי הוא חיוני ויש לשמר אותו.
[3, 2, 3, 4, -1]
אני תומך בהפחתת קצבאות הביטוח הלאומי בישראל.
[-1, -1, -1, -1, -1]
אני תומך בהגדלת קצבאות הביטוח הלאומי בישראל.
[-1, -1, 1, -1, -1]
אני סבור שפתיחת שוק הבנקאות בישראל לתחרות מחו"ל תגרום לשיפור השירות והוזלת העלויות.
[4, 2, 3, 3, 4]
אני סבור שפתיחת שוק הבנקאות בישראל לתחרות מחו"ל תגרום לפגיעה ביציבות המערכת הפיננסית.
[2, -1, -1, -1, 2]
אני